In [ ]:
from landscape_analysis_toolbox.constrained_landscape_features import (
    ConstrainedLandscapeFeatures,
)
from distribution_optimization_py.constrained_landscape_analysis import (
    DistributionOptimizationProblem,
)
from distribution_optimization_py.datasets import DATASETS, Dataset
import pandas as pd
import numpy as np

dataset_name_to_features_df: dict[str, pd.DataFrame] = {}
for dataset in DATASETS:
    problem = DistributionOptimizationProblem(
        data=dataset.data, nr_of_modes=dataset.nr_of_modes
    )
    feature_calculator = ConstrainedLandscapeFeatures(
        bounds=problem.get_bounds(),
        fitness=lambda x: problem.fitness(x),
        violation=lambda x: problem.violation(x),
        sampler=lambda sample_size, _: np.array(
            [problem.initialize() for _ in range(sample_size)]
        ),
    )
    print(dataset.name)
    feature_calculator.plot_fitness_validation()
    dataset_name_to_features_df[dataset.name] = feature_calculator.compute_features()

In [ ]:
dataset_name_to_mean_features_df = {
    name: df.mean() for name, df in dataset_name_to_features_df.items()
}

In [ ]:
all_mean_dfs = []
for name, df in dataset_name_to_mean_features_df.items():
    df.name = name
    all_mean_dfs.append(df)
all_mean_df = pd.concat(all_mean_dfs, axis=1).T

In [ ]:
all_mean_df